## C S 329E HW 9

# Crossvalidation and hyperparameter selection

## Tyler Valdez (UTEID: trv359)

For this week's homework we are going to explore the cross validation testing methodology and applying that to get error estimates on the two algorithms we've used so far:
  - Linear Regression
  - Decision Tree classification
  

In [1]:
# import the libraries! If you want to add things here for visualization, please do, 
# but only use sklearn when prompted
import pandas as pd
import numpy as np
from sklearn import tree 
from sklearn.datasets import load_iris
from sklearn.datasets import load_diabetes

# Part 1 - Calculate Generalized Error on Linear Regression with k-fold Cross Validation

## Q1.1 Load in the diabetes data set as a pandas dataframe and series.  
Documentation is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html).  Name your features dataframe (the independent variables) `df_X` and your target (the dependent variable) series `s_y`

In [2]:
df = load_diabetes()
df_X = df['data']
s_y = df['target']
print(df_X)
print(s_y)

[[ 0.03807591  0.05068012  0.06169621 ... -0.00259226  0.01990842
  -0.01764613]
 [-0.00188202 -0.04464164 -0.05147406 ... -0.03949338 -0.06832974
  -0.09220405]
 [ 0.08529891  0.05068012  0.04445121 ... -0.00259226  0.00286377
  -0.02593034]
 ...
 [ 0.04170844  0.05068012 -0.01590626 ... -0.01107952 -0.04687948
   0.01549073]
 [-0.04547248 -0.04464164  0.03906215 ...  0.02655962  0.04452837
  -0.02593034]
 [-0.04547248 -0.04464164 -0.0730303  ... -0.03949338 -0.00421986
   0.00306441]]
[151.  75. 141. 206. 135.  97. 138.  63. 110. 310. 101.  69. 179. 185.
 118. 171. 166. 144.  97. 168.  68.  49.  68. 245. 184. 202. 137.  85.
 131. 283. 129.  59. 341.  87.  65. 102. 265. 276. 252.  90. 100.  55.
  61.  92. 259.  53. 190. 142.  75. 142. 155. 225.  59. 104. 182. 128.
  52.  37. 170. 170.  61. 144.  52. 128.  71. 163. 150.  97. 160. 178.
  48. 270. 202. 111.  85.  42. 170. 200. 252. 113. 143.  51.  52. 210.
  65. 141.  55. 134.  42. 111.  98. 164.  48.  96.  90. 162. 150. 279.
  92.  83. 

## Q1.2 Define a function that creates a linear least squares regression model 
This function should take in two parameters, `df_X`, and `s_y` and return the least squares regression model, $\hat{\beta}$ (using the notation from the ESLII book chapter 3 and HW2).  You can not use any libraries outside of pandas and numpy. 

In [3]:
def get_linear_regression_model(df_X, s_y):
    [beta, residuals, rank, s] = np.linalg.lstsq(df_X, s_y, rcond=-1)
    return beta

## Q1.3 Define a function that partitions the dataframe and series data into dictionaries
This function should take in three parameters, `df_X`, `s_y`, and `k`, and returns a tuple of two dictionaries.
In both dictionaries the key is the `k` value (an integer from one to `k` inclusive).
The first dictionary will have the dataframe of the training data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable).
The second dictionary will have the series of the target data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable). Please note the targets _must match_ the same rows as the dataframe at this index, e.g, the length of the kth partition is the same for the dataframe and series.

Call that function with $k=5$ and create the dictionaries `dict_k_df_X` and `dict_k_s_y`. Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

In [4]:
def partition_data(df_X, s_y, k):
    dict_k_df_X = {}
    dict_k_s_y = {}
    cur_idx = 0
    
    for i in range(1, k):
        partition_size = int(np.round(len(df_X) / k + (np.random.normal() * len(df_X) /(k * 20))))
        dict_k_df_X[i] = df_X[cur_idx:cur_idx + partition_size]
        dict_k_s_y[i] = s_y[cur_idx:cur_idx + partition_size]
        cur_idx += partition_size
        ones = np.ones((len(dict_k_df_X[i]), 1))
        dict_k_df_X[i] = np.hstack((dict_k_df_X[i], ones))
    
    dict_k_df_X[k] = df_X[cur_idx:len(df_X)]
    dict_k_s_y[k] = s_y[cur_idx:len(df_X)]
    ones = np.ones((len(dict_k_df_X[k]), 1))
    dict_k_df_X[k] = np.hstack((dict_k_df_X[k], ones))
    
    return (dict_k_df_X, dict_k_s_y)

In [5]:
(dict_k_df_X, dict_k_s_y) = partition_data( df_X, s_y, 5 )

In [6]:
# Check sizes of folds
folds = 0

for key, values in dict_k_df_X.items():
    folds += len(values)
    print("Fold " + str(key) + " length is " + str(len(values)) + " and length of series is " + str(len(dict_k_s_y[key])))
    
print("The sum of the number of elements in the folds is " + str(folds) + " and there are " + str(len(df_X)) + " rows in the dataframe")

Fold 1 length is 86 and length of series is 86
Fold 2 length is 83 and length of series is 83
Fold 3 length is 88 and length of series is 88
Fold 4 length is 89 and length of series is 89
Fold 5 length is 96 and length of series is 96
The sum of the number of elements in the folds is 442 and there are 442 rows in the dataframe


## Q1.4 Define a function that calculates a regression metric
This function should accept two series of equal length $n$ numpy arrays, `s_y`, and `s_y_hat`. The metric it should calculate is the mean absolute error, $MAE = \sum\limits_{i=1}^n\frac{|{s\_y_i - {s\_y\_hat}_i}|}{n}$ 

Test your function by using the vectors:
```
x = np.array([1,2,3])
y = np.array([2,2,3])
```


In [8]:
def get_mae(s_y, s_y_hat):
    mae = 0 
    for i in range(len(s_y)):
        mae += abs(s_y[i] - s_y_hat[i]) / len(s_y)
    return mae

In [9]:
# Test it 
x = np.array([1,2,3])
y = np.array([2,2,3])
get_mae(x,y)

0.3333333333333333

## Q1.5 Calculate the $MAE$ for each fold
For each fold in your dictionaries, calculate the $MAE$.  Use the partition number key as the test set, and all other partitions as the train set. 

Print the min, max, and mean $MAE$ of your 5 folds. 

In [13]:
mae = np.array([])
for i in dict_k_df_X.keys():
    first_found = False
    # Iterate through the keys to make training set
    for j in dict_k_df_X.keys():
        if (i != j):
            if (first_found == False):
                training_df_X = dict_k_df_X[j]
                training_s_y = dict_k_s_y[j]
                first_found = True
            else:
                training_df_X = np.append(training_df_X, dict_k_df_X[j], axis=0)
                training_s_y = np.append(training_s_y, dict_k_s_y[j], axis=0)
    beta = np.reshape(get_linear_regression_model(training_df_X, training_s_y), (11, 1))
    s_y_hat = np.matmul(dict_k_df_X[i], beta)
    mae = np.append(mae, get_mae(dict_k_s_y[i], s_y_hat)) 
print("The min MAE is", str(mae.min()), ", the max MAE is ", str(mae.max()), "and the mean MAE is", str(mae.mean()))

The min MAE is 40.77253290622211 , the max MAE is  46.503957413558176 and the mean MAE is 44.5078559731298


# Part 2 - Find the best hyperparameter to use in a Decision Tree 

## Q2.1 Load the iris data in as a pandas dataframe and a series
Documentation is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html). Name your features dataframe (the independent variables) `df_X` and your classification target (the dependent variable) series `s_y`

In [15]:
df = load_iris()
df
df_X = df['data']
s_y = df['target']

## Q2.2 Partition `df_X` and `s_y` into $5$ partitions of roughly equal size
Make 2 dictionaries, with the key of each dictionary the fold number.  The value of the dictionary `dict_k_df_X` is the $k^{th}$ partition of the data, and the value of the dictionary `dict_k_s_y` is the corresponding $k^{th}$ target classification.  Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

In [23]:
(dict_k_df_X, dict_k_s_y) = partition_data(df_X, s_y, 5)

## Q2.3 Define a function that calculates accuracy
The function should accept two series and compare each element for equality.  The accuracy is the number of equal elements divided by the total number of elements.

Test your accuracy function by calling it with the `s_y` loaded from the iris data set and an array of the same length containing all $1$ values. 

In [24]:
def get_acc( s_1, s_2 ):
    n = len(s_1)
    c = 0.0;
    for i in range(n):
        if s_1[i] == s_2[i]:
            c += 1
    return c/n;

In [25]:
get_acc(s_y,np.ones(len(s_y)))

0.3333333333333333

## Q2.4 Using Nested Cross validation, find the best hyperparameter
Use the [Decision Tree Classifier](https://scikit-learn.org/stable/modules/tree.html#classification) class to build a decision tree inside of a 5-fold cross validation loop.  The partitions you already created in 2.2 will be the outer loop.  In the inside loop you should use 4-fold cross validation (so you don't have to partition _again_) to find the best value for `min_impurity_decrease`.  Use the Gini Index as your impurity measure. 
    Calculate the mean accuracy across the 4 folds of your inner loop for all the candidate `min_impurity_decrease` values, and print the value.  Use the array `np.array([0.1,0.25,0.3,0.4])` as the candidates for the best hyperparameter. If there is a tie (two `min_impurity_decrease` values give the same highest accuracy), choose the lowest `min_impurity_decrease` value. 

For each inner loop, select the best `min_impurity_decrease` and train the outer fold training data on using that value. 

For each inner loop, your output should look something like this:
```
Testing 0.10 min impurity decrease
	Average accuracy over 4 folds is 0.95
Testing 0.25 min impurity decrease
	Average accuracy over 4 folds is 0.86
Testing 0.30 min impurity decrease
	Average accuracy over 4 folds is 0.63
Testing 0.40 min impurity decrease
	Average accuracy over 4 folds is 0.27

Best min impurity decrease is 0.1

```

In [37]:
possible_min_impurity_decrease = np.array([0.1, 0.25, 0.3, 0.4])

# Outer loop containing partitions from 2.2
outer_acc = np.array([])
for k in dict_k_df_X.keys():
    test_df_X = dict_k_df_X[k]
    test_s_y = dict_k_s_y[k]
    first = False

    for j in dict_k_df_X.keys():
        if (k != j):
            if (first == False):
                outerFold_df_X = dict_k_df_X[j]
                outerFold_s_y = dict_k_s_y[j]
                first = True
            else:
                outerFold_df_X = np.append(outerFold_df_X, dict_k_df_X[j], axis=0)
                outerFold_s_y = np.append(outerFold_s_y, dict_k_s_y[j], axis=0)
    (dict_Inner_df_X, dict_Inner_s_y) = partition_data(outerFold_df_X, outerFold_s_y, 4)
    this_acc = np.array([])
    for pos_min_impurity in possible_min_impurity_decrease:
        inner_acc = np.array([]);
        for m in dict_Inner_df_X.keys():
            inner_test_df_X = dict_Inner_df_X[m]
            inner_test_s_y = dict_Inner_s_y[m]
            first = False
            for n in dict_Inner_df_X.keys():
                if (m!=n):
                    if (first== False):
                        innerFold_df_X = dict_Inner_df_X[n]
                        innerFold_s_y = dict_Inner_s_y[n]
                    else:
                        innerFold_df_X = np.append(innerFold_df_X, dict_Inner_df_X[n], axis=0)
                        innerFold_s_y = np.append(innerFold_s_y, dict_Inner_s_y[n], axis=0)
            
            clf = tree.DecisionTreeClassifier(min_impurity_decrease=pos_min_impurity, criterion="gini")
            clf = clf.fit(innerFold_df_X, innerFold_s_y)
            innerPrediction = clf.predict(inner_test_df_X)
            inner_acc = np.append(inner_acc, get_acc(innerPrediction, inner_test_s_y))
        print("Testing " + str(pos_min_impurity) + " min impurity decrease")
        print("Average accuracy over 4 folds is " + str(np.round(inner_acc.mean(), 2)))
        this_acc = np.append(this_acc, inner_acc.mean())
    # Inner loop cross validation code using 4 folds
    print("Best min impurity " + str(possible_min_impurity_decrease[np.argmax(this_acc)]))
    # Use best min impurity to train model
    clf = tree.DecisionTreeClassifier(min_impurity_decrease=possible_min_impurity_decrease[np.argmax(this_acc)], criterion="gini")
    clf = clf.fit(outerFold_df_X, outerFold_s_y)
    outerPrediction = clf.predict(test_df_X)
    acc = get_acc(outerPrediction,test_s_y)
    # Outer accuracy calculations
    outer_acc = np.append(outer_acc, acc) # make sure and calculate this_acc in your loop
    print("Outer accuracy " + str(np.round(acc * 100, 2)) + "%\n")

Testing 0.1 min impurity decrease
Average accuracy over 4 folds is 0.39
Testing 0.25 min impurity decrease
Average accuracy over 4 folds is 0.39
Testing 0.3 min impurity decrease
Average accuracy over 4 folds is 0.39
Testing 0.4 min impurity decrease
Average accuracy over 4 folds is 0.39
Best min impurity 0.1
Outer accuracy 100.0%

Testing 0.1 min impurity decrease
Average accuracy over 4 folds is 0.44
Testing 0.25 min impurity decrease
Average accuracy over 4 folds is 0.45
Testing 0.3 min impurity decrease
Average accuracy over 4 folds is 0.44
Testing 0.4 min impurity decrease
Average accuracy over 4 folds is 0.45
Best min impurity 0.25
Outer accuracy 100.0%

Testing 0.1 min impurity decrease
Average accuracy over 4 folds is 0.44
Testing 0.25 min impurity decrease
Average accuracy over 4 folds is 0.45
Testing 0.3 min impurity decrease
Average accuracy over 4 folds is 0.45
Testing 0.4 min impurity decrease
Average accuracy over 4 folds is 0.45
Best min impurity 0.25
Outer accuracy 0.0%

## Q2.5 Show the generalized performance of the classifier 
Show the generalized performance of the classifier by printing the min, max, and mean accuracy of the outer fold test sets.

In [40]:
print("The min accuracy is", str(outer_acc.min()), ", the max accuracy is ", str(outer_acc.max()), "and the mean accuracy is", str(outer_acc.mean()))

The min accuracy is 0.0 , the max accuracy is  1.0 and the mean accuracy is 0.7681034482758621
